In [52]:
!pip install --upgrade transformers


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/15/fc/7b6dd7e1adc0a6407b845ed4be1999e98b6917d0694e57316d140cc85484/transformers-4.39.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.2 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/05/c0/779afbad8e75565c09ffa24a88b5dd7e293c92b74eb09df6435fc58ac986/huggingface_hub-0.22.2-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<0.19,>=0.14 from https://files.pythonhosted.org/packages/5f/4f/a4c12cc058a899c1caaa1e689c3df9a698e20e891d4005aa6ec2174a9339/tokenizers-0.15.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for safetensors>=0.4.1 from https://files.pythonhosted.org/packages/03/a0/67f2ed19dbc609ce1bf561a285a6cc8f804846a7f4d9c46b403be93022e4/safetensors-0.4.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obt

In [17]:
def load_text_files(directory):
    files = os.listdir(directory)
    files = [file for file in files if not file.startswith('.DS')]
    files = sorted(files, key=lambda x: int(re.split('\.|_', x)[1]))
    print(len(files))
    data = []
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(directory, file), 'r', encoding='utf-8',errors='ignore') as f:
                text = f.read()
                data.append(text)
    return data


In [18]:
# Load text files from directories
import os
import re
original_text = load_text_files('sumpubmed-master/text')[:1000]
summaries = load_text_files('sumpubmed-master/shorter_abstract')[:1000]
print(len(original_text))
print(len(summaries))
data = {'original_text': original_text, 'summary': summaries}

32689
32689
1000
1000


In [19]:
for i in range(10):
    print(len(original_text[i]), len(summaries[i]))

40907 1875
25548 1422
32083 2235
31588 1677
22660 1968
45624 1617
47588 2070
29101 1728
17390 1551
26273 897


In [20]:
data = {
    "text": original_text,
    "summary": summaries
}


In [21]:
from datasets import Dataset

# Assuming data_dict is your initial dataset dictionary
dataset = Dataset.from_dict(data)

# Split the dataset into training and evaluation sets
train_test_split = dataset.train_test_split(test_size=0.1)  # Adjust the split ratio as needed
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']


In [105]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = 'allenai/led-base-16384'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Ignored unknown kwarg option special


In [106]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples['text'], max_length=4096, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=256, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_datasets = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

/Users/pavandukka/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [108]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,  # Start with 1 due to high memory demands; adjust as needed
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    report_to="none"  # Reduce logging verbosity if preferred
)


In [110]:
from transformers import AutoModelForSeq2SeqLM, Trainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_eval_datasets,  # Provide the evaluation dataset here
    tokenizer=tokenizer
)

In [111]:
trainer.train()

  0%|          | 0/2700 [00:00<?, ?it/s]

You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 2.8646, 'learning_rate': 4.074074074074074e-05, 'epoch': 0.56}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 2.3731348514556885, 'eval_runtime': 70.5485, 'eval_samples_per_second': 1.417, 'eval_steps_per_second': 0.184, 'epoch': 1.0}
{'loss': 2.4748, 'learning_rate': 3.148148148148148e-05, 'epoch': 1.11}
{'loss': 2.0777, 'learning_rate': 2.2222222222222223e-05, 'epoch': 1.67}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 2.318596601486206, 'eval_runtime': 59.682, 'eval_samples_per_second': 1.676, 'eval_steps_per_second': 0.218, 'epoch': 2.0}
{'loss': 1.9471, 'learning_rate': 1.2962962962962962e-05, 'epoch': 2.22}
{'loss': 1.7481, 'learning_rate': 3.7037037037037037e-06, 'epoch': 2.78}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 2.3423874378204346, 'eval_runtime': 78.9431, 'eval_samples_per_second': 1.267, 'eval_steps_per_second': 0.165, 'epoch': 3.0}
{'train_runtime': 4980.5246, 'train_samples_per_second': 0.542, 'train_steps_per_second': 0.542, 'train_loss': 2.1825097430193865, 'epoch': 3.0}


TrainOutput(global_step=2700, training_loss=2.1825097430193865, metrics={'train_runtime': 4980.5246, 'train_samples_per_second': 0.542, 'train_steps_per_second': 0.542, 'train_loss': 2.1825097430193865, 'epoch': 3.0})

In [112]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 2.3423874378204346, 'eval_runtime': 102.0456, 'eval_samples_per_second': 0.98, 'eval_steps_per_second': 0.127, 'epoch': 3.0}


In [23]:
from transformers import AutoModelForSeq2SeqLM

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained('./results/checkpoint-2500')

In [24]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('./results/checkpoint-2500')


In [25]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples['text'], max_length=4096, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=256, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/Users/pavandukka/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [26]:
from datasets import load_metric

rouge = load_metric('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = [label.split(tokenizer.pad_token, 1)[0] for label in tokenizer.batch_decode(labels, skip_special_tokens=True)]  # Remove padding

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_aggregator=True)
    return {
        "rouge1": result['rouge1'].mid.fmeasure,
        "rouge2": result['rouge2'].mid.fmeasure,
        "rougeL": result['rougeL'].mid.fmeasure,
        "rougeLsum": result['rougeLsum'].mid.fmeasure
    }


In [27]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',  # Directory to save the output results
    do_predict=True,
    per_device_eval_batch_size=2  # Adjust based on your hardware capabilities
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    eval_dataset=tokenized_eval_dataset  # Pass the tokenized evaluation dataset
)


/Users/pavandukka/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

  0%|          | 0/25 [00:00<?, ?it/s]


KeyboardInterrupt



In [50]:
# Set the model to use the MPS device
model = model.to('mps')

def generate_summary(text):
    # Ensure tensors are explicitly moved to the MPS device
    inputs = tokenizer(text, return_tensors="pt", max_length=4096, truncation=True).to('mps')
    # Adding parameters to handle padding and EOS more gracefully
    outputs = model.generate(
        inputs['input_ids'],
        max_length=100,  # Adjusted to match typical output lengths
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [51]:
# Example usage
test_text = load_text_files('sumpubmed-master/text')[1000:1100]
val_summary = load_text_files('sumpubmed-master/shorter_abstract')[1000:1100]
print(len(test_text))


32689
32689
100


In [52]:
gen_summary = []
for i in test_text:
    summary = generate_summary(i)
    gen_summary.append(summary)
    print(summary)

a number of genes and transcripts were detected to be differentially expressed in endometrium tissue on day  <dig> of gestation, during maternal recognition of pregnancy.
we identified  <dig>  differentially expressed genes and  <dig> differentially expressed transcripts in uterine endometrium tissue on day  <dig> of gestation, during maternal recognition of pregnancy.
these data contribute toward a deeper understanding of the molecular events associated with reproductive success in pigs.
the pig
in this study, two error correction packages are analyzed by examining their effects on next-generation sequencing data from a heterozygous genome.
two error correction packages are comparatively analyzed by examining their effects on next-generation sequencing data from a heterozygous genome.
two error correction packages are comparatively analyzed by examining their effects on next-generation sequencing data from a heterozygous genome.
two error correction packages are comparatively analyzed

Input ids are automatically padded from 3968 to 4096 to be a multiple of `config.attention_window`: 1024


in this study we analysed the transcriptional changes in the developing humerus and associated joints at theiler stage  23- <dig> in muscle-less  compared to phenotypically normal littermate controls.
in this study we analysed the transcriptional changes in the developing humerus and associated joints at theiler stage  23- <dig> in muscle-less  compared to phenotypically normal littermate controls.
in this study we analysed the transcriptional changes in
the aim of this study was to test the efficacy and practicability of oral fluid collection from individual pigs via cotton gauze swabs and a dry foam swab  as well as the re-collection of oral fluid from these materials.
the aim of the study was to test the efficacy and practicability of oral fluid collection from individual pigs via cotton gauze swabs and a dry foam swab  as well as the re-collection of oral fluid from these materials.

the crude enzyme was characterized for its catalytic activities and its components identified by pr

Input ids are automatically padded from 3811 to 4096 to be a multiple of `config.attention_window`: 1024


our results suggest that co-expression analysis is a powerful tool in the discovery of new targets for analysis in pathways or networks of interest.
in addition to co-expressed genes, a large number of new candidate genes were identified that are co-expressed with important genes in the sa, ja and et signaling pathways.
in addition to co-expressed genes, a large number of new candidate genes were identified that are co-expressed with important genes in the sa, ja
we analyzed wes data from a family with a history of nshl by focusing on three categories of genetic information: cnv, linkage, and snvs.
we identified  <dig>  coding autosomal snvs,  <dig>   <dig>   <dig>   <dig>   <dig>   <dig>   <dig>  and  <dig>   <dig>   <dig> 
we here show that the large intracellular domain of human pit <dig> is dispensable for pi transport function but dispensable for infection by pit <dig> cognate gamma-retroviruses.
the human pit <dig> histidine, h <dig>  located in the c-terminal pit family signatur

...

chondrodysplasia is a developmental bone defect occurring due to disturbed endochondral osteogenesis.
chondrodysplasia is a developmental bone defect occurring due to disturbed endochondral osteogenesis.
chondrodysplasia is a developmental bone defect occurring due to disturbed endochondral osteogenesis.
chondrodysplasia is a developmental bone defect occurring due to disturbed endochondral osteogenesis.
chondrodysplasiachondrodys
herpesviruses are members of herpesviridae family, a large family of dna viruses that cause chronic, latent and recurrent infections in animals and humans.
herpesviruses preferentially target pin hubs and bottlenecks, a process similar to that of human-encoded mirnas.
herpesviruses are double-stranded dna viruses with large genomes encoding complex virus particles and enzymes involved in a variety of cellular process, including nucleic acid metabolism,
in silico analysis of celtos and human interleukin  <dig> fused to human interleukin- <dig>  and m cell-spec

Input ids are automatically padded from 2538 to 3072 to be a multiple of `config.attention_window`: 1024


over-transcribed detoxification genes were found in adults of the vauclin strain, indicating a significant role of p450s and gsts in insecticide resistance in martinique.
in addition,  <dig> detoxification genes were found to be over-transcribed in larvae and adults of the vauclin strain, indicating a significant role of p450s and gsts in insecticide resistance in martinique.
in addition,  <dig>


Input ids are automatically padded from 1160 to 2048 to be a multiple of `config.attention_window`: 1024


our new normalization method does not require that the majority of genes be non-differentially expressed, and the numbers of over- and under-expressed genes are roughly equal; it appears much more effective than standard normalization methods when the numbers of over- and under-expressed genes are unequal, and the number of over- and under-expressed genes are roughly equal.
our new normalization method does not require that the majority of genes be non-differentially expressed,
single incision laparoscopic surgery  reduces surgical trauma and adhesion through implementing a small number and/or size of portals.
single incision laparoscopic surgery  could be an available, feasible and safe alternative to multiport laparoscopic surgery for splenectomy in dogs.
single incision laparoscopic surgery  could be an available, feasible and safe alternative to multiport laparoscopy for dogs undergoing elective splenectomy.
single incision lapar
in this study, we propose a "parallel" approach to m

In [55]:
!pip install rouge-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [58]:
from rouge_score import rouge_scorer
scores = []
# Initialize a Rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

for i in range(len(gen_summary)):
    scores.append(scorer.score(gen_summary[i],val_summary[i]))
print(scores)


[{'rouge1': Score(precision=0.20816326530612245, recall=0.7391304347826086, fmeasure=0.3248407643312101), 'rouge2': Score(precision=0.06967213114754098, recall=0.25, fmeasure=0.10897435897435896), 'rougeL': Score(precision=0.11836734693877551, recall=0.42028985507246375, fmeasure=0.18471337579617836)}, {'rouge1': Score(precision=0.25842696629213485, recall=0.2839506172839506, fmeasure=0.2705882352941177), 'rouge2': Score(precision=0.10227272727272728, recall=0.1125, fmeasure=0.10714285714285715), 'rougeL': Score(precision=0.23595505617977527, recall=0.25925925925925924, fmeasure=0.24705882352941178)}, {'rouge1': Score(precision=0.20080321285140562, recall=0.6493506493506493, fmeasure=0.3067484662576687), 'rouge2': Score(precision=0.06451612903225806, recall=0.21052631578947367, fmeasure=0.09876543209876543), 'rougeL': Score(precision=0.10040160642570281, recall=0.3246753246753247, fmeasure=0.15337423312883436)}, {'rouge1': Score(precision=0.21348314606741572, recall=0.6129032258064516,

In [61]:
import numpy as np

# Initialize dictionaries to store total scores and count
# Assume `data` is your list of dictionaries containing ROUGE scores

# Initialize total dictionaries
rouge1_totals = {'precision': 0, 'recall': 0, 'fmeasure': 0}
rouge2_totals = {'precision': 0, 'recall': 0, 'fmeasure': 0}
rougeL_totals = {'precision': 0, 'recall': 0, 'fmeasure': 0}

# Sum up all scores
for entry in scores:
    rouge1_totals['precision'] += entry['rouge1'].precision
    rouge1_totals['recall'] += entry['rouge1'].recall
    rouge1_totals['fmeasure'] += entry['rouge1'].fmeasure
    rouge2_totals['precision'] += entry['rouge2'].precision
    rouge2_totals['recall'] += entry['rouge2'].recall
    rouge2_totals['fmeasure'] += entry['rouge2'].fmeasure
    rougeL_totals['precision'] += entry['rougeL'].precision
    rougeL_totals['recall'] += entry['rougeL'].recall
    rougeL_totals['fmeasure'] += entry['rougeL'].fmeasure

# Calculate the number of entries
num_entries = len(scores)

# Calculate mean scores
rouge1_means = {key: value / num_entries for key, value in rouge1_totals.items()}
rouge2_means = {key: value / num_entries for key, value in rouge2_totals.items()}
rougeL_means = {key: value / num_entries for key, value in rougeL_totals.items()}

# Print the results
print("ROUGE-1 Mean Scores:", rouge1_means)
print("ROUGE-2 Mean Scores:", rouge2_means)
print("ROUGE-L Mean Scores:", rougeL_means)



ROUGE-1 Mean Scores: {'precision': 0.18888690661058077, 'recall': 0.604345182108172, 'fmeasure': 0.27875036777401224}
ROUGE-2 Mean Scores: {'precision': 0.06159820090090915, 'recall': 0.2127420660184897, 'fmeasure': 0.092803511412365}
ROUGE-L Mean Scores: {'precision': 0.11850062983307419, 'recall': 0.3838589422134849, 'fmeasure': 0.17492864837603045}


In [1]:
import os
from datasets import Dataset
import pandas as pd
import re
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
!git clone https://github.com/vgupta123/sumpubmed

Cloning into 'sumpubmed'...
remote: Enumerating objects: 130808, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 130808 (delta 22), reused 17 (delta 6), pack-reused 130764
Receiving objects: 100% (130808/130808), 346.82 MiB | 7.43 MiB/s, done.
Resolving deltas: 100% (38368/38368), done.
Updating files: 100% (130765/130765), done.


In [3]:
def load_text_files(directory):
    files = os.listdir(directory)
    files = [file for file in files if not file.startswith('.DS_Store')]
    files = sorted(files, key=lambda x: int(re.split('\.|_', x)[1]))
    data = []
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(directory, file), 'r', encoding='utf-8',errors='ignore') as f:
                text = f.read()
                data.append(text)
    return data

In [5]:
# Load text files from directories
original_text = load_text_files('sumpubmed/text')
summaries = load_text_files('sumpubmed/shorter_abstract')
# Create a dataset from the loaded text files
data = {'original_text': original_text, 'summary': summaries}

In [6]:
# Create a dataset from the loaded text files
df =pd.DataFrame({'original_text': original_text, 'summary': summaries})

# dataset = Dataset.from_dict(data)

print(df.shape)

(32689, 2)


In [7]:
import re

def remove_words_followed_by_newline(input_string):
    # Words to remove if followed by a newline character
    words_to_remove = ['BACKGROUND', 'RESULTS', 'CONCLUSIONS', 'electronic supplementary material', 'keywords']

    # Create a pattern to match the words followed by a newline
    pattern = re.compile(r'(' + '|'.join(words_to_remove) + r')(?:\n)')

    # Remove the specified words followed by a newline character
    cleaned_string = re.sub(pattern, '', input_string)

    return cleaned_string

In [8]:
def get_triples(doc):
  triples = list(textacy.extract.subject_verb_object_triples(doc))
  #print(triples)
  return triples

In [9]:
def get_nodes_relations(triples):
  nodes = []
  relations = []
# iterate over the triples
  for triple in triples:
    # extract the Subject and Object from triple
    node_subject = "_".join(map(str, triple.subject))
    node_object  = "_".join(map(str, triple.object))
    nodes.append(node_subject)
    nodes.append(node_object)
    # extract the relation between S and O
    # add the attribute 'action' to the relation
    relation = "_".join(map(str, triple.verb))
    relations.append((node_subject,node_object,{'action':relation}))
# remove duplicate nodes
  nodes = list(set(nodes))
  #print(nodes)
# ['to_extract_SVO', 'I']
  #print(relations)
# [('I', 'to_extract_SVO', {'action': 'am_going'})]
  return (nodes, relations)

In [10]:
def get_graph(nodes,relations):
  G = nx.DiGraph()
  G.add_nodes_from(nodes)
  G.add_edges_from(relations)
  return G

In [11]:
def draw_graph(G):
# extract the attribute 'action' from edges
  edge_attribute = nx.get_edge_attributes(G, 'action')
  edges, weights = zip(*edge_attribute.items())
# resize figure
  plt.rcParams["figure.figsize"] = [5, 2]
  plt.rcParams["figure.autolayout"] = True
# set figure layout
  pos = nx.circular_layout(G)
# draw graph
  nx.draw(G, pos, node_color='b', width=2, with_labels=True)
# draw edge attributes
  nx.draw_networkx_edge_labels(G, pos,edge_attribute, label_pos=0.75 )
  plt.show()

In [15]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[:1000])

In [21]:
from transformers import AutoModelForSeq2SeqLM

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained('GraphembeddedModel')

In [23]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('GraphembeddedModel')


In [29]:
# Example usage
test_text = load_text_files('sumpubmed-master/text')[1000:1100]
val_summary = load_text_files('sumpubmed-master/shorter_abstract')[1000:1100]
print(len(test_text))

100


In [32]:
test_text[0]

'BACKGROUND\nthe pig  has been used as model animal in many fields, including biological  <cit> , agricultural, and biomedical studies  <cit> . pig litter size, which can be impacted by multiple factors including ovulation rate, embryonic survival, uterine capacity and fetal survival, is one of the most important economic traits affecting production efficiency in the pig industry  <cit> . improvements in litter size across the swine industry in recent decades has occurred through different selection schemes such as phenotypic selection, family index selection, blup-based selection, component trait selection, and hyperprolific selection  <cit> . however, despite current improvements, the low heritability of litter size has limited the genetic selection progress in substantially improving this trait  <cit> . improvement of litter size is ideally suited for application of molecular breeding approaches such as marker-assisted selection  and whole-genome selection, where molecular markers a

In [36]:
# custome Dialogue Prediction

from transformers import pipeline  # Import the pipeline function
def generate_summary(text):
    pipe = pipeline('summarization', model=model, tokenizer=tokenizer)
    gen_kwargs = {'length_penalty': 0.8, 'num_beams': 8, "max_length": 1000}

    custom_dialogue=text
    print(pipe(custom_dialogue, **gen_kwargs))


    returned_summary=pipe(text,**gen_kwargs) #do_sample=False)#max_length=130, min_length=30,
    #print("Original text:\n"+text)
    #print("\n\nGenerated summary:\n"+returned_summary[0]['summary_text'])
    return returned_summary[0]['summary_text']

In [39]:
gen_summary = []
c = 0
for i in test_text:
    c+=1
    print(c)
    gen_summary.append(generate_summary(i))

1
[{'summary_text': 'a number of genes and transcripts were detected to be differentially expressed in the uterine endometrium of yorkshire and yorkshire gilts on day  <dig> of gestation.\nwe identified  <dig> differentially expressed genes in the uterine endometrium of yorkshire and yorkshire gilts on day  <dig> of gestation, and  <dig> differentially expressed genes in the uterine endometrium of yorkshire gilts on day  <dig> of gestation.\nthe uterine endometrium provides an ideal window to investigate the molecular mechanisms that may be associated early conceptus loss and sow reproductive prolificacy.\na number of genes and transcripts were detected to be differentially expressed in the uterine endometrium of yorkshire and yorkshire gilts on day  <dig> of gestation.\na number of genes and transcripts were detected to be differentially expressed in the uterine endometrium of yorkshire and yorkshire gilts on day  <dig> of gestation.\na number of genes and transcripts were detected to

Input ids are automatically padded from 14468 to 15360 to be a multiple of `config.attention_window`: 1024


Original text:
BACKGROUND
polyploidy is a particularly important evolutionary mechanism in flowering plants  <cit> . during the past  <dig> years, many plant biologists have estimated the frequency of polyploidy in the angiosperms using analysis of base chromosome numbers  <cit> , as well as measurements of stomatal size in fossil and extant taxa  <cit> . based on these approaches, researchers estimated that from 40% to 70% of angiosperms have experienced polyploidy in their evolutionary history  <cit> . recent genomic studies indicate, however, that polyploidy is even more prevalent in angiosperm lineages than previously suspected. sequencing of the entire nuclear genome of arabidopsis thaliana indicated two or three rounds of genome-wide duplication  <cit> . complete genome sequences also indicate multiple ancient polyploidy events in populus trichocarpa and vitis vinifera  <cit> . genomic data  indicate ancient polyploidy for other angiosperms  <cit> , including the basal angiosperm

Input ids are automatically padded from 1550 to 2048 to be a multiple of `config.attention_window`: 1024


Original text:
BACKGROUND
wingless and transforming growth factor beta  signaling are two cell-cell signaling systems that are well conserved throughout the animal kingdom and that control a plethora of processes ranging from embryonic development, cell proliferation, differentiation and migration, to tissue homeostasis, stem cell behavior, tissue regeneration and cancer  <cit> . dact  proteins have been identified in mammals, chicken, frog and zebrafish as intracellular multi-adapter molecules with the ability to modulate and possibly integrate the wnt and tgfβ signaling cascades. this ability primarily relies on the physical interaction of dact proteins with dvl , ckiδ/ϵ, vangl, pka, pkc, which are players in the various wnt pathways, or with the alk4/ <dig> tgfβ receptors  <cit> . in line with these properties, dact proteins positively as well as negatively regulate the wnt/β-catenin pathway and positively regulate the wnt/pcp pathway . in addition, specifically dact <dig> has been 

...

Original text:
BACKGROUND
burkholderia cenocepacia j <dig> is a member of the burkholderia cepacia complex , a group of  <dig> species of closely related gram-negative betaproteobacteria  <cit>  which occur in the soil rhizosphere and also play an important role as opportunistic pathogens in humans . bcc bacteria are intrinsically resistant to most antibiotics, and infections with bcc bacteria are therefore difficult to treat. bcc bacteria are also able to form biofilms, further adding to their recalcitrance to antibiotic treatment  <cit> .

b. cenocepacia j <dig> harbours a large  <dig>  mb multi-replicon genome with a high average gc-content of  <dig>  %. the genome consists of two large replicons of  <dig>  and  <dig>  mb, a smaller replicon  <dig>  mb and a plasmid  <dig>  mb in size, with  <dig> annotated protein coding and  <dig> annotated rna genes  <cit> , including  <dig> trnas and  <dig> riboswitches. however, transcription start sites , 5′ untranslated regions  of annotated 

Input ids are automatically padded from 4750 to 5120 to be a multiple of `config.attention_window`: 1024


Original text:
BACKGROUND
agriculture, in struggling to meet global food demands of a rapidly growing population, has been plagued with plant diseases that thwart crop production and account for a global annual average yield loss of  <dig> percent . in light of the importance of agriculture to humans, it is essential to elucidate and understand the mechanisms of plant immunity at the molecular level. as with any host-pathogen conflict, plants and their disease agents are in an evolutionary arms race. when the host mounts a defense reaction, the pathogen develops new strategies to evade the defensive mechanisms, which causes the continuation of this cycle, ad infinitum <cit> . our present understanding of plant immune systems reveals two primary means by which plants recognize their invaders. the recognition of microbial-associated molecular patterns  is the first line of defense and its associated mamps-triggered immunity  is highly efficient at repelling most pathogens. if pathogens a

Input ids are automatically padded from 4424 to 5120 to be a multiple of `config.attention_window`: 1024


Original text:
BACKGROUND
candida albicans colonize distinct microanatomical regions such as the oro-gastointestinal tract as a commensal, but also accounts for more than 50% of all fungal systemic infections  <cit> . candida esophagitis represents a severe threat to an immunocompromised body and, especially in neutropenic patients, often is the first manifestation before candida sepsis develops  <cit> . host defense mechanisms preventing mucosal  candidiasis are poorly understood, but include both innate and adaptive responses  <cit> .

the gastrointestinal epithelial layer represents a barrier that is usually adequate to restrain commensal microbes, but is often insufficient to protect against microbial pathogens. once this physical barrier is penetrated, recognition of invading microbiota is the first step in the initiation of a fast immune response and involves the activation of pattern recognition receptors by microbial pathogens and their products  <cit> . c. albicans have been s

In [40]:
from rouge_score import rouge_scorer
scores = []
# Initialize a Rouge scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

for i in range(len(gen_summary)):
    scores.append(scorer.score(gen_summary[i],val_summary[i]))
print(scores)


[{'rouge1': Score(precision=0.3346938775510204, recall=0.47953216374269003, fmeasure=0.3942307692307692), 'rouge2': Score(precision=0.0942622950819672, recall=0.13529411764705881, fmeasure=0.1111111111111111), 'rougeL': Score(precision=0.20816326530612245, recall=0.2982456140350877, fmeasure=0.24519230769230768)}, {'rouge1': Score(precision=0.29213483146067415, recall=0.12440191387559808, fmeasure=0.174496644295302), 'rouge2': Score(precision=0.03409090909090909, recall=0.014423076923076924, fmeasure=0.020270270270270268), 'rougeL': Score(precision=0.25842696629213485, recall=0.11004784688995216, fmeasure=0.15436241610738255)}, {'rouge1': Score(precision=0.1566265060240964, recall=0.34210526315789475, fmeasure=0.21487603305785125), 'rouge2': Score(precision=0.028225806451612902, recall=0.061946902654867256, fmeasure=0.038781163434903045), 'rougeL': Score(precision=0.10843373493975904, recall=0.23684210526315788, fmeasure=0.1487603305785124)}, {'rouge1': Score(precision=0.17977528089887

In [42]:
import numpy as np
import pandas as pd

# Create lists to hold the data for DataFrame
precision1, recall1, fmeasure1 = [], [], []
precision2, recall2, fmeasure2 = [], [], []
precisionL, recallL, fmeasureL = [], [], []

# Populate lists with data
for entry in scores:
    precision1.append(entry['rouge1'].precision)
    recall1.append(entry['rouge1'].recall)
    fmeasure1.append(entry['rouge1'].fmeasure)
    precision2.append(entry['rouge2'].precision)
    recall2.append(entry['rouge2'].recall)
    fmeasure2.append(entry['rouge2'].fmeasure)
    precisionL.append(entry['rougeL'].precision)
    recallL.append(entry['rougeL'].recall)
    fmeasureL.append(entry['rougeL'].fmeasure)

# Create a DataFrame
df = pd.DataFrame({
    'ROUGE-1 Precision': precision1,
    'ROUGE-1 Recall': recall1,
    'ROUGE-1 F-measure': fmeasure1,
    'ROUGE-2 Precision': precision2,
    'ROUGE-2 Recall': recall2,
    'ROUGE-2 F-measure': fmeasure2,
    'ROUGE-L Precision': precisionL,
    'ROUGE-L Recall': recallL,
    'ROUGE-L F-measure': fmeasureL
})

# Save the DataFrame to an Excel file
df.to_excel('rouge_scores.xlsx', index=False)


In [ ]:
import numpy as np

# Initialize dictionaries to store total scores and count
# Assume `data` is your list of dictionaries containing ROUGE scores

# Initialize total dictionaries
rouge1_totals = {'precision': 0, 'recall': 0, 'fmeasure': 0}
rouge2_totals = {'precision': 0, 'recall': 0, 'fmeasure': 0}
rougeL_totals = {'precision': 0, 'recall': 0, 'fmeasure': 0}

# Sum up all scores
for entry in scores:
    rouge1_totals['precision'] += entry['rouge1'].precision
    rouge1_totals['recall'] += entry['rouge1'].recall
    rouge1_totals['fmeasure'] += entry['rouge1'].fmeasure
    rouge2_totals['precision'] += entry['rouge2'].precision
    rouge2_totals['recall'] += entry['rouge2'].recall
    rouge2_totals['fmeasure'] += entry['rouge2'].fmeasure
    rougeL_totals['precision'] += entry['rougeL'].precision
    rougeL_totals['recall'] += entry['rougeL'].recall
    rougeL_totals['fmeasure'] += entry['rougeL'].fmeasure

# Calculate the number of entries
num_entries = len(scores)

# Calculate mean scores
rouge1_means = {key: value / num_entries for key, value in rouge1_totals.items()}
rouge2_means = {key: value / num_entries for key, value in rouge2_totals.items()}
rougeL_means = {key: value / num_entries for key, value in rougeL_totals.items()}

# Print the results
print("ROUGE-1 Mean Scores:", rouge1_means)
print("ROUGE-2 Mean Scores:", rouge2_means)
print("ROUGE-L Mean Scores:", rougeL_means)

